In [91]:
import pandas as pd

import os

In [92]:
def caminho_absoluto(caminho_relativo_com_barras_normais):
    
    caminho_base = os.getcwd()

    caminho_absoluto = os.path.join(caminho_base, caminho_relativo_com_barras_normais)

    return caminho_absoluto

In [93]:
situacao_final = pd.read_csv(caminho_absoluto('data/tratamento_curva_abc/dados_tratados/csv/situacao_final.csv'))

In [94]:
situacao_final.head(5)

,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,...,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Frac.,Tipo,Qtde Desvio Picking,local
0,106011.0,NARIDRIN 12 HORAS FR C/30ML,A,A,A,302581.0,5910.0,893581.0,278.0,251.0,...,66,2.0,100.0,170.0,100.0,Não,29.913,Ponta De G,4725.0,pallet
1,231012.0,EXPEC XAROPE FR C/120ML,A,A,A,139507.0,5293.0,457087.0,274.0,215.0,...,47,2.0,60.0,170.0,240.0,Não,26.025,Flowrack,3232.0,pallet
2,101190.0,EMSEXPECTOR XPE 120ML,A,A,A,131640.0,974.0,190080.0,279.0,171.0,...,56,2.0,60.0,214.0,240.0,Não,21.911,Ponta De G,2519.0,pallet
3,102011.0,CYSTEX C/ 24DRG,A,A,A,120667.0,220.0,142667.0,285.0,48.0,...,18,20.0,100.0,246.0,300.0,Não,29.063,Flowrack,5361.0,apanha_a
4,101001.0,APEVITIN BC XPE 240ML,A,A,A,111539.0,1721.0,171774.0,286.0,211.0,...,51,2.0,35.0,122.0,105.0,Não,29.911,Ponta De G,4637.0,pallet


In [95]:
selecao_dividir = ((situacao_final['Estoque Frac'] != 0) & \
                   (situacao_final['Tipo'] == 'Prateleira') & \
                   (situacao_final['Ender.Frac.'] > 20) & \
                   (situacao_final['Ender.Frac.'] < 28)
                   )

prateleira_para_dividir = situacao_final[selecao_dividir]

In [96]:
selecao_nao_e_end_do_meio = ~((prateleira_para_dividir['Ender.Frac.'] > 21.580) & \
                              (prateleira_para_dividir['Ender.Frac.'] < 22) | \
                              (prateleira_para_dividir['Ender.Frac.'] > 22.580) & \
                              (prateleira_para_dividir['Ender.Frac.'] < 23)
                              )

prateleira_para_dividir = prateleira_para_dividir[selecao_nao_e_end_do_meio]

In [97]:
def comeca_com(df, corredor):
    return df['Ender.Frac.'].astype(str).str.startswith(corredor)

In [98]:
# Mais saida
ir_para_final_modulo = prateleira_para_dividir[0:768]

selecao_trocar_para_o_final = ((comeca_com(ir_para_final_modulo, '20')) | \
                               (comeca_com(ir_para_final_modulo, '23')) | \
                               (comeca_com(ir_para_final_modulo, '24')) | \
                               (comeca_com(ir_para_final_modulo, '27'))
                               )

colocar_nos_corredores_21_22_25_26 = ir_para_final_modulo[selecao_trocar_para_o_final]

In [99]:
print(len(colocar_nos_corredores_21_22_25_26))
colocar_nos_corredores_21_22_25_26.head(3)

364


,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,...,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Frac.,Tipo,Qtde Desvio Picking,local
42,63591.0,"COMPRESSA HERIKA 13 FIOS 7,5CMX7,5 C/10",A,C,A,23466.0,0.0,23466.0,38.0,0.0,...,2 (+344 fr),2.0,700.0,50.0,50.0,Sim,24.144,Prateleira,228.0,apanha_b
242,63628.0,"FITA CIRUR MICROPOROSA BRANCA 2,5CMX4,5M",A,C,A,3762.0,1.0,3858.0,56.0,1.0,...,1,1.0,96.0,47.0,45.0,Não,24.265,Prateleira,60.0,prateleira
259,63986.0,"COMPRESSA HÉRIKA 9FIO 7,5X7,5CM26CM C/10",A,NaN,A,3441.0,NaN,3441.0,12.0,NaN,...,0 (+608 fr),1.0,1020.0,20.0,20.0,Sim,20.203,Prateleira,324.0,apanha_b


In [103]:
colocar_nos_corredores_21_22_25_26.to_excel('colocar_nos_corredores_21_22_25_26.xlsx', index=False)

In [100]:
ir_para_comeco_modulo = prateleira_para_dividir[768:]

selecao_trocar_para_o_comeco = ((comeca_com(ir_para_comeco_modulo, '21')) | \
                                (comeca_com(ir_para_comeco_modulo, '22')) | \
                                (comeca_com(ir_para_comeco_modulo, '25')) | \
                                (comeca_com(ir_para_comeco_modulo, '26'))
                                )

colocar_nos_corredores_20_23_24_27 = ir_para_comeco_modulo[selecao_trocar_para_o_comeco]

In [101]:
print(len(colocar_nos_corredores_20_23_24_27))
colocar_nos_corredores_20_23_24_27.head(3)

236


,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,...,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Frac.,Tipo,Qtde Desvio Picking,local
2076,12661.0,ACECLOFENACO 100MG C/ 12COMP,C,NaN,C,82.0,NaN,82.0,21.0,NaN,...,0,1.0,48.0,76.0,48.0,Não,25.282,Prateleira,11.0,apanha_c
2077,19145.0,AREMAZ 7MG/ML SOL ORAL C/ 100 ML-MLB,C,NaN,C,82.0,NaN,82.0,15.0,NaN,...,0,2.0,60.0,29.0,30.0,Sim,26.285,Prateleira,7.0,apanha_c
2078,610623.0,NAXOTEC 500MG 24COMP,C,NaN,C,82.0,NaN,82.0,18.0,NaN,...,0,1.0,198.0,34.0,36.0,Sim,25.227,Prateleira,4.0,apanha_c


In [102]:
colocar_nos_corredores_20_23_24_27.to_excel('colocar_nos_corredores_20_23_24_27.xlsx', index=False)